In [1]:
!pip install refractml==1.0.3
# !pip install scikit-learn==0.24.2

     |████████████████████████████████| 51kB 1.2MB/s eta 0:00:011
     |████████████████████████████████| 665kB 1.1MB/s eta 0:00:01
     |████████████████████████████████| 143kB 4.0MB/s eta 0:00:01
     |████████████████████████████████| 61kB 3.9MB/s eta 0:00:01
     |████████████████████████████████| 71kB 3.8MB/s eta 0:00:01
     |████████████████████████████████| 71kB 2.9MB/s eta 0:00:01
     |████████████████████████████████| 51kB 3.7MB/s eta 0:00:01
     |████████████████████████████████| 9.6MB 4.4MB/s eta 0:00:01
     |████████████████████████████████| 143kB 5.8MB/s eta 0:00:01
     |████████████████████████████████| 71kB 5.4MB/s eta 0:00:01
     |████████████████████████████████| 163kB 5.7MB/s eta 0:00:01
     |████████████████████████████████| 18.3MB 5.6MB/s eta 0:00:01
     |████████████████████████████████| 307kB 7.6MB/s eta 0:00:01
     |████████████████████████████████| 38.5MB 4.3MB/s eta 0:00:01
  Created wheel for shutils: filename=shutils-0.1.0-cp39-none-any.whl size=3283

In [2]:
!pip uninstall urllib3 -y

Uninstalling urllib3-1.26.15:
  Successfully uninstalled urllib3-1.26.15


In [ ]:
!pip install urllib3==1.26.15

In [1]:
import pandas as pd
import numpy as np
# import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import requests
from refractml import *
from refractml.constants import MLModelFlavours

/opt/conda/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [2]:
cars = pd.read_csv("car_new.csv")
cars

,buying_num,maintenance_num,doors_num,persons_num,lugboot_num,safety_num,class_num
0,3,3,0,0,2,2,2
1,3,3,0,0,2,0,2
2,3,3,0,0,1,1,2
3,3,3,0,0,1,2,2
4,3,3,0,0,1,0,2
...,...,...,...,...,...,...,...
1722,1,1,3,2,1,2,1
1723,1,1,3,2,1,0,3
1724,1,1,3,2,0,1,2
1725,1,1,3,2,0,2,1


In [4]:
Y = cars['class_num']
X = cars.drop (['class_num'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.18, shuffle=True, random_state=25)
print (X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1416, 6) (311, 6) (1416,) (311,)


In [5]:
first_tree = DecisionTreeClassifier()
model=first_tree.fit(X_train, y_train)
model

DecisionTreeClassifier()

In [6]:
y_pred = model.predict(X_test)
first_tree.score(X_test, y_test)
y_prob = model.predict_proba(X_test)[:,1]

In [7]:
@scoring_func
def score(model, request):
    payload = request.json["payload"]
    data = pd.DataFrame(eval(payload))
    prediction = pd.Series(model.predict(data))
    return prediction.tolist()

In [8]:
req = requests.Request()
req.json = {"payload":X_train.head(1).to_json()}

sch = generate_schema(score,(model, req), X_train)
print(sch)

{'input': {'type': 'pd.core.frame.DataFrame', 'items': {'type': 'object', 'properties': {'buying_num': {'type': 'integer', 'format': 'int64'}, 'maintenance_num': {'type': 'integer', 'format': 'int64'}, 'doors_num': {'type': 'integer', 'format': 'int64'}, 'persons_num': {'type': 'integer', 'format': 'int64'}, 'lugboot_num': {'type': 'integer', 'format': 'int64'}, 'safety_num': {'type': 'integer', 'format': 'int64'}}}, 'example': [{'buying_num': 1, 'maintenance_num': 2, 'doors_num': 0, 'persons_num': 2, 'lugboot_num': 0, 'safety_num': 1}, {'buying_num': 3, 'maintenance_num': 2, 'doors_num': 2, 'persons_num': 2, 'lugboot_num': 2, 'safety_num': 2}]}, 'output': {'type': 'array', 'items': {'type': 'object'}, 'example': [2]}}


In [9]:
!pip show scikit-learn

Name: scikit-learn
Version: 1.3.2
Summary: A set of python modules for machine learning and data mining
Home-page: http://scikit-learn.org
Author: None
Author-email: None
License: new BSD
Location: /packages/Python-3.8/de678bea-42c4-42fe-88b0-04bfdecd1b35/3.8
Requires: joblib, numpy, scipy, threadpoolctl
Required-by: TPOT, mlflow, mosaic-utils


In [10]:
register_model(model, 
               score, 
               name='Refractml_Sklearn_Python_38', 
               description='car evaluation classification model using sklearn', 
               flavour=MLModelFlavours.sklearn,
               schema=sch,
               y_true=y_test, 
               y_pred=y_pred,
               y_train=y_train, 
               y_test=y_test,
               x_train=X_train, 
               x_test=X_test, 
               prob=y_prob,
               labels=[0,1], 
               init_script="pip install scikit-learn==1.3.2",
               model_type="classification",
               input_type="json", 
               explain_ai=False, 
               feature_names=list(X_train), 
               feature_ids=list(X_train),
               target_names=[0,1,2,3], 
               kyd=True, 
               kyd_score = False
              )

Calculating build time metrics

Progress: ██████████████████████████████████████████████████████████████████████ 100.0%
